In [1]:
import json
import pandas as pd
import requests
import tqdm

from io import StringIO
from multiprocessing.pool import Pool

# Materials specific packages
from ase.io import read as ase_read

# MDF Utils
from mdf_dataworks.toolbox import login, gmeta_pop

# Authentication
To authenticate a client, pass the app name, service name, and the index to search to globus_auth.login 


In [2]:
config = {
    "app_name": "mdf_examples",
    "services": ["search"],
    "index": "mdf"
}
client_results = login(credentials=config)
client = client_results["search"]

# Perform Queries

### List all MDF datasets

In [3]:
q = {
    "q": "mdf.resource_type:dataset",
    "advanced": True,
    "limit": 9999
}

res = gmeta_pop(client.structured_search(q))
mdf_resources = [(r['mdf']['source_name'],r['mdf']['title']) for r in res]
df = pd.DataFrame(mdf_resources, columns=['source_name','title'])

print("Number of data resources: {n_datasets}".format(n_datasets=len(df)))
df.head(15)

Number of data resources: 83


,source_name,title
0,jcap_xps_spectral_db,JCAP XPS Spectral Database
1,gw100,Benchmark of G0W0 on 100 Molecules
2,gw_soc81,Benchmark of G0W0 on 81 Molecules with Spin-Or...
3,nist_th_ar_lamp_spectrum,NIST Spectrum of Th-Ar Hollow Cathode Lamps
4,nist_heat_transmission,NIST Heat Transmission Properties of Insulatin...
5,core_mof,Computation-Ready Experimental Metal-Organic F...
6,nanomine,NanoMine
7,bursi_toxicophores,Derivation and Validation of Toxicophores for ...
8,xafs_sl,XAFS Spectra Library
9,dilute_mg_alloys_dft,Data set for diffusion coefficients of alloyin...


### Finding a Specific Dataset

In [4]:
q = {
    "q": "mdf.source_name:oxygen_interstitials_deformation AND mdf.resource_type:dataset",
    "advanced": True
}

res = client.structured_search(q)
print(json.dumps(res['gmeta'][0], sort_keys=True,indent=4, separators=(',', ': ')))

{
    "@datatype": "GMetaResult",
    "@version": "2016-11-09",
    "content": [
        {
            "mdf": {
                "author": [
                    {
                        "email": "william.joost@gmail.com",
                        "family_name": "Joost",
                        "full_name": "William Joost",
                        "given_name": "William",
                        "institution": "University of Maryland"
                    },
                    {
                        "family_name": "Ankem",
                        "full_name": "Sreeramamurthy Ankem",
                        "given_name": "Sreeramamurthy",
                        "institution": "University of Maryland"
                    },
                    {
                        "family_name": "Kuklja",
                        "full_name": "Maija Kuklja",
                        "given_name": "Maija",
                        "institution": "University of Maryland"
                    }
         

### Get Records for a dataset

In [5]:
q = {
    "q": "mdf.source_name:oxygen_interstitials_deformation AND mdf.resource_type:record",
    "advanced": True,
}

res = client.structured_search(q)
print("Total Matching Records:"+str(res['total']))
print(json.dumps(res['gmeta'][0:5], sort_keys=True,indent=4, separators=(',', ': ')))

Total Matching Records:15
[
    {
        "@datatype": "GMetaResult",
        "@version": "2016-11-09",
        "content": [
            {
                "mdf": {
                    "collection": "Oxygen Interstitials and Deformation Twins in alpha-Titanium",
                    "composition": "OTi160",
                    "elements": [
                        "O",
                        "Ti"
                    ],
                    "ingest_date": "2017-07-11T15:55:18.685844Z",
                    "links": {
                        "landing_page": "https://materialsdata.nist.gov/dspace/xmlui/handle/11256/272#1",
                        "outcar": {
                            "globus_endpoint": "82f1b5c6-6e9b-11e5-ba47-22000b92c6ec",
                            "http_host": "https://data.materialsdatafacility.org",
                            "path": "/collections/oxygen_interstitials_deformation/0L_VASP/ShortCell_0L_C1_DFT/OUTCAR"
                        },
                       

### Get records for a given dataset with a specific element in the composition

In [6]:
source_name = "core_mof"
element = "Cl"

q = {
    "q": "mdf.source_name:{source_name} AND \
          mdf.resource_type:record AND mdf.elements:{element}".format(source_name = source_name, 
                                                                      element=element),
    "advanced": True,
    "limit":20
}

res = client.structured_search(q)
print("Total Matching Records:"+str(res['total']))
print(json.dumps(res['gmeta'][0:5], sort_keys=True,indent=4, separators=(',', ': ')))

Total Matching Records:116
[
    {
        "@datatype": "GMetaResult",
        "@version": "2016-11-09",
        "content": [
            {
                "mdf": {
                    "collection": "CoRE-MOF",
                    "composition": "C24Cl8O16Th2",
                    "elements": [
                        "Cl",
                        "O",
                        "C",
                        "Th"
                    ],
                    "ingest_date": "2017-07-11T14:59:19.436720Z",
                    "links": {
                        "cif": {
                            "http_host": "https://raw.githubusercontent.com",
                            "path": "/gregchung/gregchung.github.io/master/CoRE-MOFs/core-mof-v1.0-ddec/MOBBOU_clean.cif"
                        },
                        "landing_page": "https://github.com/gregchung/gregchung.github.io/blob/master/CoRE-MOFs/core-mof-v1.0-ddec/MOBBOU_clean.cif",
                        "parent_id": "5964e765d0a0d30b759